In [154]:

from Point_Er import Point
from Lattice_Er import Lattice
from Simulator_Er import Simulator
from utils import *
from EnergyTransfer_Er import *
import numpy as np



In [155]:
tag_default={'c0':1e-39, # Yb-Yb resonant energy transfer
        'Ws': 639,
 'E1E0': 88.12753083306136,
 'E2E1': 13.728308752002313,
 'E2E0': 105.00663885847584,
 
 'E3E2': 0.6904748414556272,
 'E3E1': 40.06626483314129,
 'E3E0': 107.07825106403719,

 'E4E3': 1.4142534182563467,
 'E4E2': 49.124834957391506,
 'E4E1': 45.114305779338295,
 'E4E0': 1009.6221517188111,

 'E5E4': 0.5491077883920105 + 3909/3.5,
 'E5E3': 46.481404188403104,
 'E5E2': 28.889483458690968,
 'E5E1': 378.15231194559027,
 'E5E0': 919.1044353717751,

 'E6E5': 0.02617036285192619 + 14290358.698538769/3.5,
 'E6E4': 8.841624545216064,
 'E6E3': 47.60084543949401,
 'E6E2': 41.09061870168263,
 'E6E1': 71.35702052573745,
 'E6E0': 2812.803587953125,

 'E7E6': 0.4092613138830535 + 2927313.4554100684/3.5,
 'E7E5': 0.01904121955274265,
 'E7E4': 3.4467583618029134,
 'E7E3': 93.44157758618482,
 'E7E2': 162.98778196545229,
 'E7E1': 334.1120016219258,
 'E7E0': 2256.2758284193}

# Progress file

In [156]:
import os
import pickle
from datetime import datetime
from time import sleep


# load or initialize progress
def load_or_initialize_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    else:
        return {}

# save progress
def save_progress(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

progress_folder = 'Chuanyu_data_files'
if not os.path.exists(progress_folder):
    os.makedirs(progress_folder)

# unique filename
current_date = datetime.now().strftime('%m_%d_%Y')
base_filename = f'Progress_{current_date}'
index = 1
progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')
    
while os.path.exists(progress_filepath):
    index += 1
    progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')


# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_progress.pkl'
# if there is no at least one combination finished, there would be no 'myC_progress.pkl' in the directory
# check the progress data if the program is interruptted


"""
file_path = 'Chuanyu_data_files/myC_progress.pkl'

with open(file_path, 'rb') as file:
    progress_data = pickle.load(file)

for key, value in progress_data.items():
    print(key, value)
"""

# Load or initialize progress data
# if progress_file_path doesn't exist, then return a {}
# if progress_file_path exists, then return a progress_data, which is an incomplete myC
# after interruption, run code and the the progress bar will start from the next combination
# after finish, the progress file is large: ~60M

myC = load_or_initialize_progress(progress_filepath)


# Instruction file

In [157]:
import os
import nbformat as nbf
from datetime import datetime

def create_instruction_notebook(guide_folder='Chuanyu_data_files', saturation_plot_file='SaturationCurves.py', optimal_percentage_file='OptimalPercentage.py', population_evolution_file='PopulationEvolution.py'):
    """
    Create a Jupyter Notebook with instructions and code to load data and generate plots.
    
    Parameters:
    - guide_folder (str): Folder where the guide notebook will be saved.
    - saturation_plot_file (str): Path to the Python file containing the SaturationPlot class.
    - optimal_percentage_file (str): Path to the Python file containing the SinglePowerDensityPlot class.
    - population_evolution_file (str): Path to the Python file containing the PopulationEvolutionPlot class.
    """

    if not os.path.exists(guide_folder):
        os.makedirs(guide_folder)

    # unique filename
    current_date = datetime.now().strftime('%m_%d_%Y')
    base_filename = f'Guide_{current_date}'
    index = 1
    notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')
    while os.path.exists(notebook_path):
        index += 1
        notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')

    # quick notes
    instructions = input("Please provide instructions for this dataset: ")

    # create a new ipynb object
    nb = nbf.v4.new_notebook()

    # add a markdown cell with the provided instructions
    nb.cells.append(nbf.v4.new_markdown_cell(f"# Guide for Dataset - {current_date}_{index}\n\n{instructions}"))

    # add a code cell for data loading
    nb.cells.append(nbf.v4.new_code_cell(f"""\
import pickle
import os


# Load the data

filepath = f'myC_{current_date}_{index}.pkl'
with open(filepath, 'rb') as f:
    data = pickle.load(f)

print("Data loaded successfully")
"""))
    



    # Create the first new code cell for checking percentages and power densities
    new_cell_1 = """\
# check the percentages and power density

percentages = sorted(data.keys())
power_densities = sorted({k for subdict in data.values() for k in subdict.keys()})
print('Percentages:', percentages)
print('Power Densities:', sorted(power_densities))
    """

    # Append the first new cell to the notebook
    nb.cells.append(nbf.v4.new_code_cell(new_cell_1))

    # Create the second new code cell for calculating the ratio
    new_cell_2 = """\
ratio = {}
for p in power_densities:
    ratio[(0.02, p)] = (data[0.02][p]['green_avg'], data[0.02][p]['red_avg'], data[0.02][p]['green_red_ratio'])
    ratio[(0.6, p)] = (data[0.6][p]['green_avg'], data[0.6][p]['red_avg'], data[0.6][p]['green_red_ratio'])
ratio
    """

    nb.cells.append(nbf.v4.new_code_cell(new_cell_2))



    # add a code cell for importing and using the SaturationPlot class, SinglePowerDensityPlot class, PopulationEvolutionPlot class
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SaturationPlot class from the SaturationCurves module
from {os.path.splitext(saturation_plot_file)[0]} import SaturationPlot

# Generate the saturation curves plot
plot = SaturationPlot(data)
plot.generate_plot(output_file='saturation_plot.html')
"""))

    # add 
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SinglePowerDensityPlot class from the OptimalPercentage module
from {os.path.splitext(optimal_percentage_file)[0]} import SinglePowerDensityPlot


# Provide a list of available power densities
available_power_densities = sorted({{k for subdict in data.values() for k in subdict.keys()}})
print(f'Available power densities: {{available_power_densities}}')

# Prompt the user to select a power density
selected_power_density = float(input(f'Input a power density from the above options: '))

# Generate the single power density plot
single_plot = SinglePowerDensityPlot(data, selected_power_density)
single_plot.generate_plot(output_file='single_power_density_plot.html')
"""))

    # add 
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the PopulationEvolutionPlot class from the PopulationEvolution module
from {os.path.splitext(population_evolution_file)[0]} import PopulationEvolutionPlot

# Provide a list of available percentages

available_percentages = sorted(data.keys())
print(f'Available percentages: {{available_percentages}}')
# Prompt the user to select a percentage
percentage = float(input(f'Input a percentage from the above options: '))

available_power_densities = sorted({{k for k in data[percentage].keys()}})
print(f'Available power densities for percentage {{percentage}}: {{available_power_densities}}')
# Prompt the user to select a power density
power_density = float(input(f'Input a power density from the above options: '))




# Generate the population evolution plot
pop_plot = PopulationEvolutionPlot(data, percentage, power_density)
pop_plot.generate_plot(output_file='population_evolution_plot.html')
"""))

    # Write the notebook to the file
    with open(notebook_path, 'w') as f:
        nbf.write(nb, f)

    print(f"Instruction notebook has been successfully saved to '{notebook_path}'")
    print()


    


In [158]:
create_instruction_notebook('Chuanyu_data_files', 'SaturationCurves_Er.py', 'OptimalPercentage_Er.py', 'PopulationEvolution_Er.py')

Instruction notebook has been successfully saved to 'Chuanyu_data_files/Guide_07_31_2024_3.ipynb'



# Save file

In [159]:
# use pickle module to save the results as a pickle file and reload them later
# The / character cannot be used in file names because it is reserved as a directory separator, replace / with underscores _ or using another safe delimiter.


import pickle
from datetime import datetime
import os
import plotly.graph_objects as go
import nbformat as nbf



class PickleSaver:

    def __init__(self, base_name='Mydata', folder='Chuanyu_data_files'):
     
        self.base_name = base_name
        self.folder = folder


        if not os.path.exists(self.folder):
            os.makedirs(self.folder)

    def save_data(self, data):
      

        current_date = datetime.now().strftime('%m_%d_%Y')
        base_filename = f'{self.base_name}_{current_date}'
        
        index = 1
        filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')
        while os.path.exists(filename):
            index += 1
            filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')


        with open(filename, 'wb') as f:
            pickle.dump(data, f)

        print(f"Data has been successfully saved to '{filename}'")
        print()


    def load_all_data(self):
        
        if not os.path.exists(self.folder):
            print(f"Folder '{self.folder}' does not exist.")
            return {}

        files = [f for f in os.listdir(self.folder) if f.endswith('.pkl')]
        all_data = {}

        for file in sorted(files):
            filepath = os.path.join(self.folder, file)
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
                all_data[file] = data

        print(f'Filename = {file}')

        KEY1=[]
        KEY2=[]

        for key1, value1 in data.items():
            KEY1.append(key1)
        print(f'\nAll percentages = {KEY1}')

        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                KEY2.append(key2)
            break
        print(f'\nFor each percentage, all power densities = {KEY2}')

        # use a set to keep track of unique third keys
        unique_third_keys = set()

        print('\nFor each combination of Percentage+Power density, the accessible data: ')
        print()
        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                for key3 in value2.keys():
                    if key3 not in unique_third_keys:
                        unique_third_keys.add(key3)
                        print(f'key3 = {key3}, value3 = {value2[key3]}')

# Execution

Yuxuan_size: 4, 8, 12, 16, 20, 24, 28

Chuanyu_size: 6, 10, 14, 18, 22, 26, 30



In [160]:



# nanoparticle diameter: 8 nm, critical distance: 1 nm
d=8
shell=0
r0=1
defects_fraction = 0.1


# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# Er_conc=[0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# Yb_conc=0.2

# Er_conc = [0.06, 0.08, 0.1]
# power_density =  [1*10**3, 1*10**4, 1*10**5, 1*10**6]

# emitter_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# power_density = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 7.5*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]




# Er_conc = [0.2]
# power_density = [100000, 1000000, 10000000]

# Er_conc = [0.02, 0.6]
# power_density = [1000, 10000, 100000, 1000000, 10000000]


Er_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
d_range = [8.0, 8.3, 8.0, 8.5, 8.0, 7.9, 8.2, 8.7]
power_density = [1232.21547757,  1869.76441851,  2826.08782992,  4419.96018226,
 7607.70488696, 10795.44959165, 13983.19429634, 17170.93900104,
 20358.68370573, 23546.42841042, 26734.17311511, 29921.91781981]



t1 = 3000
t2 = 10000


# Simulate and track progress

for i, c in enumerate(Er_conc):
    if c not in myC:
        myC[c] = {}

    d=d_range[i]
    print(d)

    for p in power_density:
        if p not in myC[c]:
            print(f"Running simulation for Er concentration {c}, power density {p}")

            # sigma from MC paper, SI 
            tag_default['laser'] = 0.058 * p
            tag_default['laser_er'] = 0.0084 * p       
            
            my_lattice = Lattice(1-c, c, d, defects_fraction, r0)  # shell
            my_lattice.introduce_defects()
            
            my_simulator = Simulator(my_lattice, tag=tag_default)
            result = my_simulator.simulate(t1, t2)  # r is a dictionary from 'sim_stats'

            # Store result and save progress
            myC[c][p] = result
            # store the current progress to the filepath
            save_progress(progress_filepath, myC)

            # Simulate time delay to represent real simulation time
            sleep(1)

print('\nAll progress have been finished.')

8.0
Running simulation for Er concentration 0.04, power density 1232.21547757


  0%|          | 0/3401 [00:00<?, ?it/s]

100%|██████████| 7000/7000 [00:20<00:00, 336.68it/s]
/Users/liuchuan/Desktop/‼️My local GitHub/from newest code/My-Current-project/Simulator_Er.py:299: RuntimeWarning: invalid value encountered in scalar divide
  'green_red_ratio': (np.mean(greens)*10**6)/(np.mean(reds)*10**6),
/Users/liuchuan/Desktop/‼️My local GitHub/from newest code/My-Current-project/Simulator_Er.py:310: RuntimeWarning: invalid value encountered in scalar divide
  'red_green_ratio_pop': (np.mean(er_state_evolution[4][t1:]) * self.tag['E4E0']


Running simulation for Er concentration 0.04, power density 1869.76441851


100%|██████████| 7000/7000 [00:24<00:00, 282.56it/s]


...

Running simulation for Er concentration 0.5, power density 26734.17311511


100%|██████████| 7000/7000 [00:49<00:00, 142.62it/s]


Running simulation for Er concentration 0.5, power density 29921.91781981


100%|██████████| 7000/7000 [04:14<00:00, 27.47it/s]  



All progress have been finished.


In [153]:
# after all the progress are finished
# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_1.pkl', which store all data
# The second time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_2.pkl', which store all data

saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)



Data has been successfully saved to 'Chuanyu_data_files/myC_07_31_2024_2.pkl'



In [19]:
# then, call the method with myC, to see the data structure
data_structure = saver.load_all_data()

Filename = myC_07_25_2024_4.pkl

All percentages = [0.02, 0.6]

For each percentage, all power densities = [1000, 10000, 100000, 1000000, 10000000]

For each combination of Percentage+Power density, the accessible data: 

key3 = red_microsecond, value3 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

---
### If you want to run the code again, firstly make sure you have run the code above to store the data:

'''
saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)b
all_save_data = saver.load_all_data()
'''
#### In the same directory, there would be a new file: myC_05_08_2024_1, which save all data from this simulation
#### Then you can detele the progress file and to run the code
---

In [151]:
0.844/1.04

0.8115384615384614

In [ ]:

 {(0.02, 10000): (750.0, 666.6666666666666, 1.125),
 (0.6, 10000): (19583.333333333336, 17333.333333333332, 1.1298076923076925),
 (0.02, 10000): (583.3333333333334, 583.3333333333334, 1.0),
 (0.6, 10000): (14916.666666666666, 17333.333333333332, 0.8605769230769231),
 (0.02, 10000): (500.0, 916.6666666666666, 0.5454545454545455),
 (0.6, 10000): (16666.666666666668, 17083.333333333332, 0.9756097560975611),
 (0.02, 10000): (666.6666666666666, 583.3333333333334, 1.1428571428571428),
 (0.6, 10000): (16583.333333333332, 16500.0, 1.005050505050505),
 (0.02, 10000): (416.6666666666667, 583.3333333333334, 0.7142857142857143),
 (0.6, 10000): (16916.666666666668, 17500.0, 0.9666666666666668),
 (0.02, 10000): (916.6666666666666, 500.0, 1.8333333333333333),
 (0.6, 10000): (16083.333333333336, 16666.666666666668, 0.9650000000000001),
 (0.02, 10000): (666.6666666666666, 833.3333333333334, 0.7999999999999999),
 (0.6, 10000): (16750.0, 19083.333333333336, 0.8777292576419213),
 (0.02, 10000): (500.0, 916.6666666666666, 0.5454545454545455),
 (0.6, 10000): (15833.333333333334, 19250.0, 0.8225108225108225),
(0.02, 10000): (500.0, 250.0, 2.0),
 (0.6, 10000): (15916.666666666666, 15583.333333333332, 1.0213903743315509),
 (0.02, 10000): (666.6666666666666, 916.6666666666666, 0.7272727272727273),
 (0.6, 10000): (15666.666666666666, 17583.333333333332, 0.8909952606635071)}



 (0.02, 100000): (5666.666666666667, 6333.333333333333, 0.8947368421052633),
 (0.6, 100000): (165333.33333333334, 190250.0, 0.8690319754708717),
 (0.02, 100000): (6250.0, 6250.0, 1.0),
 (0.6, 100000): (163000.0, 192416.6666666667, 0.8471199653529665),
 (0.02, 100000): (5916.666666666666, 6250.0, 0.9466666666666665),
 (0.6, 100000): (166666.66666666666, 197083.33333333334, 0.8456659619450316),
 (0.02, 100000): (4833.333333333334, 7083.333333333333, 0.6823529411764707),
 (0.6, 100000): (169250.0, 185833.3333333333, 0.9107623318385651),
 (0.02, 100000): (5750.0, 6666.666666666667, 0.8624999999999999),
 (0.6, 100000): (160250.0, 184083.3333333333, 0.8705296514259847),
 (0.02, 100000): (5333.333333333333, 6250.0, 0.8533333333333333),
 (0.6, 100000): (162500.0, 192083.3333333333, 0.8459869848156183),
 (0.02, 100000): (6833.333333333334, 6416.666666666667, 1.064935064935065),
 (0.6, 100000): (166083.33333333334, 180583.33333333334, 0.9197046608214121),
 (0.02, 100000): (6166.666666666667, 6583.333333333333, 0.9367088607594938),
 (0.6, 100000): (164250.0, 184750.0, 0.8890392422192152),
 (0.02, 100000): (6833.333333333334, 7833.333333333333, 0.872340425531915),
 (0.6, 100000): (168583.33333333334, 187750.0, 0.897913892587661),
 (0.02, 100000): (5500.0, 5500.0, 1.0),
 (0.6, 100000): (167166.66666666666, 192166.6666666667, 0.8699045967042497),



 (0.02, 1000000): (9333.333333333334, 10833.333333333334, 0.8615384615384616),
 (0.6, 1000000): (284916.6666666666, 332416.6666666667, 0.8571070443720229),
 (0.02, 1000000): (8166.666666666666, 10583.333333333334, 0.7716535433070865),
 (0.6, 1000000): (292166.6666666667, 340083.3333333334, 0.8591031609899534),
 (0.02, 1000000): (9416.666666666666, 10666.666666666666, 0.8828125),
 (0.6, 1000000): (276833.3333333333, 335500.0, 0.8251366120218578),
 (0.02, 1000000): (9333.333333333334, 9833.333333333332, 0.9491525423728815),
 (0.6, 1000000): (298500.0, 353333.3333333333, 0.844811320754717),
 (0.02, 1000000): (8416.666666666666, 10250.0, 0.8211382113821137),
 (0.6, 1000000): (276333.3333333333, 349166.6666666667, 0.7914081145584725),
 (0.02, 1000000): (10666.666666666666, 13000.0, 0.8205128205128205),
 (0.6, 1000000): (299083.3333333333, 344916.6666666666, 0.8671176612708384),
 (0.02, 1000000): (8166.666666666666, 10583.333333333334, 0.7716535433070865),
 (0.6, 1000000): (295083.3333333333, 352166.6666666667, 0.8379081874112635),
 (0.02, 1000000): (8916.666666666666, 11833.333333333332, 0.7535211267605634),
 (0.6, 1000000): (292166.6666666667, 339250.0, 0.8612134610660772),
 (0.02, 1000000): (8833.333333333334, 10833.333333333334, 0.8153846153846154),
 (0.6, 1000000): (300666.6666666667, 347333.3333333333, 0.8656429942418428),
 (0.02, 1000000): (7500.0, 10000.0, 0.75),
 (0.6, 1000000): (285250.0, 344500.0, 0.8280116110304789),


In [142]:
(1.125+1+0.5454545454545455+1.1428571428571428+0.7142857142857143+1.8333333333333333+0.799999999999999+0.5454545454545455+2+0.7272727272727273)/10

1.0433658008658007

In [143]:
(1.1298076923076925+0.8605769230769231+0.9756097560975611+1.005050505050505+0.9666666666666668+0.9650000000000001+0.8777292576419213+0.8225108225108225+1.0213903743315509+0.8909952606635071)/10

0.9515337258347151

In [144]:
data = {
    (0.02, 100000): [(5666.666666666667, 6333.333333333333, 0.8947368421052633),
                     (6250.0, 6250.0, 1.0),
                     (5916.666666666666, 6250.0, 0.9466666666666665),
                     (4833.333333333334, 7083.333333333333, 0.6823529411764707),
                     (5750.0, 6666.666666666667, 0.8624999999999999),
                     (5333.333333333333, 6250.0, 0.8533333333333333),
                     (6833.333333333334, 6416.666666666667, 1.064935064935065),
                     (6166.666666666667, 6583.333333333333, 0.9367088607594938),
                     (6833.333333333334, 7833.333333333333, 0.872340425531915),
                     (5500.0, 5500.0, 1.0)],
    (0.6, 100000): [(165333.33333333334, 190250.0, 0.8690319754708717),
                    (163000.0, 192416.6666666667, 0.8471199653529665),
                    (166666.66666666666, 197083.33333333334, 0.8456659619450316),
                    (169250.0, 185833.3333333333, 0.9107623318385651),
                    (160250.0, 184083.3333333333, 0.8705296514259847),
                    (162500.0, 192083.3333333333, 0.8459869848156183),
                    (166083.33333333334, 180583.33333333334, 0.9197046608214121),
                    (164250.0, 184750.0, 0.8890392422192152),
                    (168583.33333333334, 187750.0, 0.897913892587661),
                    (167166.66666666666, 192166.6666666667, 0.8699045967042497)]
}

def average_third_value(data, key):
    third_values = [entry[2] for entry in data[key]]
    return sum(third_values) / len(third_values)

# Calculate the averaged third value for each key
averaged_values = {key: average_third_value(data, key) for key in data}

print(averaged_values)


{(0.02, 100000): 0.9113574134508207, (0.6, 100000): 0.8765659263181576}
